# Einführung Tensorflow/Keras

## Keras sequential model API

Das sequentielle Model ist **ein linearer Stapel** von Schichten (von Neuronen).

**Import**
```python
from keras.models import Sequential
from keras.layers import Dense, Activation
```

Man kann ein sequentielles Model erstellen, indem man an den Konstruktor eine **Liste der Schichten** übergibt, die man nutzen will:
```python
model = Sequential([
    Dense(32, input_dim=784), 
    Activation('linear'),
])
```

* `Dense()` ist eine "dichte" Schicht, jedes Neuron ist mit jedem Neuron der folgenden Schicht verbunden
  * `Dense(32)` bedeutet, dass diese Schicht aus 32 Neuronen besteht
  * `input_dim=784` ist die Spezifizierung der Eingabeform
    * bedeutet, die Eingabe soll ein 784-dimensionaler Vektor sein
* `Activation()` ist die Angabe der Aktivierungsfunktion der Neuronen (Erinnerung an die Treppenfunktion)
  * `Activation('linear')` ist die lineare Aktivierungsfunktion

### Alternative Schreibweise (add-Funktion)
Wir können das Gleiche erreichen durch die `.add()` - Funktion:
```python
model = Sequential()
model.add(Dense(32, input_dim=784))
model.add(Activation('linear'))
```

### Model konfigurieren und kompilieren

**`compile` - Funktion:**
* Konfiguration des Models
  * Wir **müssen** einen **Optimizer** angeben (sonst kann das Netz nicht lernen).
  * auch eine Fehlerfunktion und eine Metrik (Messen der Genauigkeit) können wir angeben
  * *Die weiteren Parameter brauchen wir an dieser Stelle nicht.*
  * *Fortgeschrittene Teilnehmer finden deren Definition hier:* [Keras Dokumentation](https://keras.io/models/model/#compile)

```python
model.compile(optimizer, loss=None, metrics=None, loss_weights=None, sample_weight_mode=None, weighted_metrics=None, target_tensors=None)
```

### Model trainieren

**`fit` - Funktion:**
* Training des Models
    * Wir übergeben 
        * die **Eingabedaten** (Array oder Liste)
        * die **Ziel-Daten** (gewünschtes Ergebnis pro Eintrag)
    * Die *Batch-Größe* beschreibt wie viele Einträge druch das Netz gefüttert werden, bevor der Gradient für die nächste Anpassung berechnet wird.
    * Eine *Epoche* ist eine vollständige Iteration über den gesamten Datensatz (alle Batches). Wir geben an, wie viele davon durchgeführt werden sollen.

```python
model.fit(x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None, validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None, validation_freq=1, max_queue_size=10, workers=1, use_multiprocessing=False)
```


### Model evaluieren

**`evaluate` - Funktion:**
* Evaluierung des Models
    * Gibt die Fehler- und Genauigkeitsmetriken des Models zurück
    * Berechnung jeweils pro Batch
    * Wir übergeben (wie in der `fit` Funktion)
      * die **Eingabedaten** (in diesem Falle die Trainingsdaten als Array oder Liste)
      * die **Ziel-Daten** (gewünschtes Ergebnis pro Eintrag)

```python
model.evaluate(x=None, y=None, batch_size=None, verbose=1, sample_weight=None, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False)
```


### Model trifft Vorhersagen

**`predict` - Funktion:**
* Vorhersagungen des Models
  * Vorhersage der Labels der Testdaten (noch nicht gesehene Daten)
  * Berechnung erfolgt wieder auf Batches (deren Größe wir angeben sollten).

```python
model.predict(x, batch_size=None, verbose=0, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False)
```

## Keras functional model API

Die Keras functional API bietet mehr Freiheiten in der **Definition komplexer neuronaler Netze**.

Die functional API wird in der Praxis häufiger genutzt als die sequential model API, da diese mehr für einfache Netze geeignet ist.

### Beispiel an einem vollständig verbundenem Netzwerk

*Eigentlich wäre für dieses einfache Netz die Sequential model API vollkommen ausreichend, aber an so einem Beispiel kann man die API besser verstehen.*

Eine `layer`-Instanz ist aufrufbar auf einem `tensor`. Die Rückgabe ist ein Eingabetensor und ein Ausgabetensor. Das kann genutzt werden, um ein Modell zu definieren, genau wie in der *sequentiall model API*.

**Import:**
```python
from keras.layers import Input, Dense
from keras.models import Model
```
**Eingabe definieren:**
```python
# gibt Tensor zurück
inputs = Input(shape=(784,))
```
**`layer` Instanzen aufrufen**:
* können hier beliebige Schichten kombinieren

```python
# eine Schicht ist aufrufbar auf einem tensor und gibt einen tensor zurück
output_1 = Dense(64, activation='linear')(inputs)
output_2 = Dense(64, activation='linear')(output_1)
predictions = Dense(10, activation='linear')(output_2)
```
**Model erstellen:**
```python
# Erstellt ein Model mit einer Eingabeschicht und drei "Dense" Schichten
model = Model(inputs=inputs, outputs=predictions)
```
**Model konfigurieren**:
```python
model.compile(optimizer='', loss='', metrics=[''])
```
**Model trainieren**:

```python
model.fit(data, labels)  # beginnt Training
```


### Wiederverwendung von Models
Alle `model` sind aufrufbar, genau wie `layer`

Dadurch ist es mit der functional API sehr einfach möglich Modele wiederzuverwenden:
* ein `model` kann wie eine `layer` behandelt werden und auf einem `tensor` aufgerufen werden

**Merke:** Man verwendet nicht nur den Aufbau des Models, sondern auch die Gewichte!

```python
x = Input(shape=(784,))
y = model(x)
```

**Vorteil:** Man kann sehr schnell Modelle bauen, die verschiedene Eingaben entgegennehmen.